In [1]:
import numpy as np
import pandas as pd

In [2]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

In [3]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 1. Efron's bootstrap

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
kf = KFold(n_splits=10, shuffle=True)

In [5]:
for (train, test), i in zip(kf.split(X, y), range(kf.n_splits)):
    print(len(test))

36
36
36
36
35
35
35
35
35
35


In [6]:
def rubin():
    # results - [fold[prediction[y_hat, y]]]
    results = np.empty((kf.n_splits, 36, 2))
    forests = []

    for (train, test), i in zip(kf.split(X, y), range(kf.n_splits)):
        rf = RandomForestRegressor(criterion='absolute_error', max_depth=15, max_features=1, min_samples_split=1, n_estimators=1100, random_state=42).fit(X[train], y[train])
        forests.append(rf)

        for j, covars in enumerate(X[test]):
            results[i][j][0] = rf.predict(covars.reshape(1, -1))[0]
            results[i][j][1] = y[test][j]

    return results, forests

In [12]:
def get_rubin_scores(results):
    # get MSE, MAE, r2
    scores = np.empty((10, 3))
    for i in range(0, 10):

        # exclude last number in array if evaluating folds 5-10
        if i >= 4:
            means = results[i, :-1, 0]
            targets = results[i, :-1, 1]
        else:
            means = results[i, :, 0]
            targets = results[i, :, 1]

        scores[i][0] = mean_squared_error(targets, means)
        scores[i][1] = mean_absolute_error(targets, means)
        scores[i][2] = r2_score(targets, means)

    return scores

In [8]:
def write_rubin_results(results):
    with open("boston_rf_efron_results.txt", "a") as file:
        file.write("WARNING: FOLDS 5-10 (INC.) ONLY HAVE 35 TRUE PREDICTIONS - IGNORE PREDICTION 36 \n")
        for fold in results:
            for prediction in fold:
                for i, value in enumerate(prediction):
                    file.write(str(value))

                    if i != 1:
                        file.write(", ")
                file.write("\n")
            file.write("\n")

In [9]:
def write_rubin_scores(scores):
    with open("boston_rf_efron_scores.txt", "a") as file:
        file.write("WARNING: FOLDS 5-10 (INC.) ONLY HAVE 35 TRUE PREDICTIONS - IGNORE PREDICTION 36 \n")
        for score_array in scores:
            for i, score in enumerate(score_array):
                file.write(str(score))

                if i != 2:
                    file.write(", ")
            file.write("\n")

In [15]:
def validation_rubin(forest_model, v_X, v_y):
    # results - [prediction[y_hat, y]]
    results = np.empty((len(v_y), 2))

    for i, covars in enumerate(v_X):
         results[i][0] = forest_model.predict(covars.reshape(1, -1))[0]
         results[i][1] = v_y[i]

    scores = np.empty(3)
    means = results[:, 0]

    scores[0] = mean_squared_error(v_y, means)
    scores[1] = mean_absolute_error(v_y, means)
    scores[2] = r2_score(v_y, means)

    return results, scores

repeated 10 times:

In [34]:
results, forests = rubin()
scores = get_rubin_scores(results)
write_rubin_results(results)
write_rubin_scores(scores)
print(scores)

[[ 7.85290119  1.95821212  0.86310243]
 [16.91560263  2.38084722  0.77428075]
 [16.56968812  3.03310227  0.82583832]
 [17.42240375  2.84957955  0.80707405]
 [39.46714758  3.41345325  0.59772383]
 [17.43036994  3.19532208  0.74790531]
 [ 9.7641902   2.39231948  0.82513373]
 [15.33735261  2.6507039   0.83728315]
 [13.89333502  2.56317922  0.87494693]
 [19.97418803  3.11744935  0.80997661]]


In [35]:
vr, vs = validation_rubin(forests[0], X_test, y_test) # insert best model
with open("boston_efron_validation_scores.txt", "a") as file:
    for i, score in enumerate(vs):
        file.write(str(score))
        if i != 2:
            file.write(", ")
    file.write("\n")

## 2. Rubin's bootstrap

In [36]:
from uncertainty_analysis import bootstrap

In [37]:
n_rubin_samples = 1100

In [38]:
def bf():
    # results - [fold[prediction[y_hat, b_mean, b_std, y]]]
    results = np.empty((kf.n_splits, 36, 4))

    forests = []

    for (train, test), i in zip(kf.split(X, y), range(kf.n_splits)):
        rf = RandomForestRegressor(criterion='absolute_error', max_depth=15, max_features=1, min_samples_split=1, n_estimators=1100, random_state=42).fit(X[train], y[train])
        forests.append(rf)

        # get prediction of each tree in RF
        predictions = np.empty(rf.n_estimators)

        for j, covars in enumerate(X[test]):
            for k, tree in enumerate(rf.estimators_):
                predictions[k] = tree.predict(covars.reshape(1, -1))[0]

            bootstrapped_predictions = bootstrap.bayesian_bootstrap(predictions, n_rubin_samples)

            results[i][j][0] = predictions.mean()
            results[i][j][1] = np.mean(bootstrapped_predictions)
            results[i][j][2] = np.std(bootstrapped_predictions)
            results[i][j][3] = y[test][j]

    return results, forests

In [48]:
def write_bf_results(results):
    with open("boston_rf_rubin_raw_results.txt", "a") as file:
        file.write("WARNING: FOLDS 5-10 (INC.) ONLY HAVE 35 TRUE PREDICTIONS - IGNORE PREDICTION 36 \n")
        for fold in results:
            for prediction in fold:
                for i, value in enumerate(prediction):
                    file.write(str(value))

                    if i != 3:
                        file.write(", ")
                file.write("\n")
            file.write("\n")

In [49]:
def write_bf_scores(scores):
    with open("boston_rf_rubin_scores.txt", "a") as file:
        file.write("WARNING: FOLDS 5-10 (INC.) ONLY HAVE 35 TRUE PREDICTIONS - IGNORE PREDICTION 36 \n")
        for score_array in scores:
            for i, score in enumerate(score_array):
                file.write(str(score))

                if i != 2:
                    file.write(", ")
            file.write("\n")

In [42]:
def get_bf_scores(results):
    # get MSE, MAE, r2
    scores = np.empty((10, 3))
    for i in range(0, 10):

        # exclude last number in array if evaluating folds 5-10
        if i >= 4:
            means = results[i, :-1, 1]
            targets = results[i, :-1, 3]
        else:
            means = results[i, :, 1]
            targets = results[i, :, 3]

        scores[i][0] = mean_squared_error(targets, means)
        scores[i][1] = mean_absolute_error(targets, means)
        scores[i][2] = r2_score(targets, means)

    return scores

In [46]:
def validation_bf(forest_model, v_X, v_y, n_samples):
     # results - [prediction[y_hat, b_mean, b_std, y]]
    results = np.empty((len(v_y), 4))
    predictions = np.empty(forest_model.n_estimators)

    for i, covars in enumerate(v_X):
        for j, tree in enumerate(forest_model.estimators_):
            predictions[j] = tree.predict(covars.reshape(1, -1))[0]

        bootstrapped_predictions = bootstrap.bayesian_bootstrap(predictions, n_samples)

        results[i][0] = predictions.mean()
        results[i][1] = np.mean(bootstrapped_predictions)
        results[i][2] = np.std(bootstrapped_predictions)
        results[i][3] = v_y[i]

    scores = np.empty(3)
    means = results[:, 0]

    scores[0] = mean_squared_error(v_y, means)
    scores[1] = mean_absolute_error(v_y, means)
    scores[2] = r2_score(v_y, means)

    return results, scores

repeated 10 times:

In [66]:
results, bf_forests = bf()
scores = get_bf_scores(results)
write_bf_results(results)
write_bf_scores(scores)
print(scores)

[[15.26583666  2.72634036  0.82951388]
 [ 8.51861711  2.07215503  0.89388928]
 [ 9.7461105   2.49206666  0.8132262 ]
 [19.40184179  3.08107163  0.80438933]
 [16.81060616  2.87648823  0.83604872]
 [40.79218174  4.04525033  0.61995153]
 [ 8.6567737   2.08808261  0.88791818]
 [12.47479054  2.80760972  0.85283206]
 [35.58697706  3.50792136  0.67592493]
 [11.66438361  2.43829119  0.81356408]]


In [67]:
vr, vs = validation_bf(bf_forests[1], X_test, y_test, n_rubin_samples) # insert best model
with open("boston_bf_validation_scores.txt", "a") as file:
    for i, score in enumerate(vs):
        file.write(str(score))
        if i != 2:
            file.write(", ")
    file.write("\n")

## 3. Proper Bayesian Bootstrap (Galvani et al.)

In [78]:
from uncertainty_analysis import proper_bayesian_forest as pbf

In [69]:
# create prior distributions (in this case, uniform)
def create_priors(X):
    n_features = X.shape[1]
    priors = np.empty(n_features, dtype=pbf.UniformPrior)
    for i in range(0, n_features):
        priors[i] = pbf.UniformPrior(X[:, i])

    return priors

In [77]:
def proper_bf(k):
    # create forest and train for each
    results = np.empty((kf.n_splits, 36, 3)) # [fold[prediction[mean, std, y]]]

    # store forests
    forests = []

    for (train, test), i in zip(kf.split(X, y), range(kf.n_splits)):
        pbrgr = pbf.ProperBayesianForest(X[train], y[train],
                                         create_priors(X[train]),
                                         k_values=np.array([k]*X[train].shape[1], dtype=np.int32),
                                         n_galvani_samples=1100,
                                         criterion='friedman_mse', max_depth=15, max_features=1,
                                         min_samples_split=1, n_estimators=1100, random_state=42)
        pbrgr.fit()
        forests.append(pbrgr)

        for j, covars in enumerate(X[test]):
            p = pbrgr.get_prediction_distribution(covars.reshape(1, -1),
                                                   n_rubin_samples=n_rubin_samples)

            results[i][j][0] = np.mean(p)
            results[i][j][1] = np.std(p)
            results[i][j][2] = y[test][j]

    return results, forests

In [71]:
def get_pbf_scores(results):
    # get MSE, MAE, r2
    scores = np.empty((10, 3))
    for i in range(0, 10):

        # exclude last number in array if evaluating folds 5-10
        if i >= 4:
            means = results[i, :-1, 0]
            targets = results[i, :-1, 2]
        else:
            means = results[i, :, 0]
            targets = results[i, :, 2]

        scores[i][0] = mean_squared_error(targets, means)
        scores[i][1] = mean_absolute_error(targets, means)
        scores[i][2] = r2_score(targets, means)

    return scores

In [72]:
def write_pbf_results(results):
    with open("boston_rf_pbf_raw_results.txt", "a") as file:
        file.write("WARNING: FOLDS 5-10 (INC.) ONLY HAVE 35 TRUE PREDICTIONS - IGNORE PREDICTION 36 \n")
        for fold in results:
            for prediction in fold:
                for i, value in enumerate(prediction):
                    file.write(str(value))

                    if i != 2:
                        file.write(", ")
                file.write("\n")
            file.write("\n")

In [73]:
def write_pbf_scores(scores):
    with open("boston_rf_pbf_scores.txt", "a") as file:
        file.write("WARNING: FOLDS 5-10 (INC.) ONLY HAVE 35 TRUE PREDICTIONS - IGNORE PREDICTION 36 \n")
        for score_array in scores:
            for i, score in enumerate(score_array):
                file.write(str(score))

                if i != 2:
                    file.write(", ")
            file.write("\n")

In [74]:
def validation_pbf(forest_model, v_X, v_y, n_samples):
    results = np.empty((len(v_y), 3)) # [prediction[mean, std, y]]

    for i, covars in enumerate(v_X):
        p = forest_model.get_prediction_distribution(covars.reshape(1, -1), n_rubin_samples=n_samples)

        results[i][0] = np.mean(p)
        results[i][1] = np.std(p)
        results[i][2] = v_y[i]

    scores = np.empty(3)
    means = results[:, 0]

    scores[0] = mean_squared_error(v_y, means)
    scores[1] = mean_absolute_error(v_y, means)
    scores[2] = r2_score(v_y, means)

    return results, scores

### Get PBF of multiple w values:

w = 0. Repeated 10 times

In [99]:
k1 = pbf.get_k(0, len(X[train]))
results1, pbf_forests1 = proper_bf(k1)
scores1 = get_pbf_scores(results1)
write_pbf_results(results1)
write_pbf_scores(scores1)
print(scores1)

[[13.39707401  2.74669588  0.81916745]
 [ 8.8223852   1.9149519   0.88128583]
 [11.32071564  2.6304001   0.83301736]
 [33.62470299  3.67746093  0.7327604 ]
 [13.37959967  2.61454414  0.85036109]
 [10.11277955  2.50492822  0.85276165]
 [14.46681162  2.88694895  0.88031746]
 [ 8.38668065  2.08045858  0.78986795]
 [32.8849825   3.10647828  0.46612501]
 [12.918694    2.63036437  0.87958695]]


In [98]:
vr, vs = validation_pbf(pbf_forests1[4], X_test, y_test, n_rubin_samples)
with open("boston_pbf_validation_scores_w0.txt", "a") as file:
    for i, score in enumerate(vs):
        file.write(str(score))
        if i != 2:
            file.write(", ")
    file.write("\n")

w = 0.25. Repeated 10 times

In [119]:
k2 = pbf.get_k(0.25, len(X[train]))
results2, pbf_forests2 = proper_bf(k2)
scores2 = get_pbf_scores(results2)
write_pbf_results(results2)
write_pbf_scores(scores2)
print(scores2)

[[89.16688738  6.30922833  0.18796449]
 [62.15012569  5.66906661  0.3981006 ]
 [74.06911595  5.44112212  0.27800207]
 [62.28785785  4.90028141  0.15825007]
 [60.59821156  5.34508658  0.36266539]
 [75.39526814  6.190449    0.36584643]
 [38.56737717  4.64028451  0.34088918]
 [55.50849514  5.84427705  0.42271801]
 [41.90742345  4.32310392  0.39338639]
 [27.54669271  4.40217857  0.2660341 ]]


In [120]:
vr, vs = validation_pbf(pbf_forests2[9], X_test, y_test, n_rubin_samples)
with open("boston_pbf_validation_scores_w025.txt", "a") as file:
    for i, score in enumerate(vs):
        file.write(str(score))
        if i != 2:
            file.write(", ")
    file.write("\n")

w = 0.05. Repeated 5 times

In [132]:
k3 = pbf.get_k(0.05, len(X[train]))
results3, pbf_forests3 = proper_bf(k3)
scores3 = get_pbf_scores(results3)
write_pbf_results(results3)
write_pbf_scores(results3)
print(scores3)

[[16.00820322  2.86334185  0.68101554]
 [22.65758487  3.9948171   0.58570946]
 [53.23805949  5.32317009  0.44194744]
 [27.29337799  4.0234779   0.62431924]
 [69.54749346  5.20365364  0.31471958]
 [35.63452359  4.73153201  0.5824749 ]
 [53.31109147  4.74876688  0.59743056]
 [36.95553483  4.70377152  0.5926248 ]
 [44.95841244  4.96976898  0.59478938]
 [20.87576885  2.80424151  0.6814735 ]]


In [133]:
vr, vs = validation_pbf(pbf_forests3[0], X_test, y_test, n_rubin_samples)
with open("boston_pbf_validation_scores_w005.txt", "a") as file:
    for i, score in enumerate(vs):
        file.write(str(score))
        if i != 2:
            file.write(", ")
    file.write("\n")